In [ ]:
!pip install gradio transformers torch

Modules require for AI generation.


In [ ]:
from transformers import pipeline
import spacy
import gradio as gr
import torch
import warnings
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.stats import shapiro
import warnings

warnings.filterwarnings('ignore')

Building the model


In [ ]:
classifier = pipeline("zero-shot-classification", model = 'facebook/bart-large-mnli')

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/515 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Function for differentiating types of prompt


In [ ]:
def classify_prompt(prompt):
  labels = ['descriptive_statistics', 'correlation', 'regression', 'others']
  result = classifier(prompt, candidate_labels = labels)
  return result['labels'][0]

Function for extracting multiple variables from prompt


In [ ]:
nlp = spacy.load("en_core_web_sm")

def extract_variables(prompt):
    doc = nlp(prompt)
    variables = [ent.text for ent in doc.ents if ent.label_ == "GPE" or ent.label_ == "ORG"]  # Customize for variables
    if len(variables) == 0:
        return None
    return variables

Function for Loading CSV file and Selecting columns for analysis


In [ ]:
def load_file(file):
  if not file.name.endswith('.csv'):
    raise ValueError('File must be a CSV file')
  return pd.read_csv(file.name)
def df_column(df):
  return df.columns.tolist()

Functions that does the analysis (Descriptive, Correlation and regression)


In [ ]:
def descriptive_statistics(df, variables):
  return df[variables].describe()
def correlation(df, variables):
  return df[variables].corr()
def normal_distribution(df, variables):
  for variable in variables:
    stat, p = shapiro(df[variable])
    return stat, p
def regression(df, dep_variables, ind_variables):
  X = df[ind_variables]
  y = df[dep_variables]
  model = LinearRegression()
  model.fit(X, y)
  return model.coef_, model.intercept_


Function that does the analysis


In [ ]:
def analyse_data(file, prompt):
  analysis_type = classify_prompt(prompt)
  variables = extract_variables(prompt)
  if variables is None:
    raise ValueError('No variables found in prompt')
  try:
    df = load_file(file)
  except ValueError as e:
    return str(e)
  colums = df_column(df)
  if set(variables).issubset(set(colums)):
    if analysis_type == 'descriptive_statistics':
      result = descriptive_statistics(df, variables)
    elif analysis_type == 'correlation':
      result = correlation(df, variables)
    elif analysis_type == 'regression':
      result = regression(df, variables[0], variables[1:])
    elif analysis_type == 'normal_distribution':
      result = normal_distribution(df, variables)
    else:
      result = 'Other analysis type'

  else:
    raise ValueError('Variables not found in file')
  return result

Function that shows the Graphycal user interface

In [ ]:
interface = gr.Interface(fn=analyse_data,
                         inputs=[gr.File(label="Upload CSV File"), gr.Textbox(label="Analysis Prompt")],
                         outputs="text",
                         title = "Gen AI Data Analysis Assistant")

interface.launch(debug = True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://296b1c9c50da047308.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 416, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1138, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://296b1c9c50da047308.gradio.live
